In [1]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F
import lightning as L
from lightning.pytorch.loggers import WandbLogger
from scipy import stats
from sklearn.preprocessing import LabelEncoder
import esm
import dask as dd

# log into to wandb to log results
import wandb

WANDB_NOTEBOOK_NAME = '2024-06-13_mbe-linear.ipynb'

wandb.login()

datadir = '../out/corrected/counts'
procdir = '../out/modelling/processed'
BATCH_SIZE=64

os.makedirs(procdir, exist_ok=True)
np.random.seed(12345)
rerun_encoding = False


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tomy-lorant (lorant). Use `wandb login --relogin` to force relogin


## Data import
Load the r0 and r1 datasets.

In [2]:
# get files to be imported
files = glob.glob(os.path.join(datadir, '2024-06-05_r[0-1]_np-cc*aa-seq-counts.tsv.gz'))
files

['../out/corrected/counts/2024-06-05_r0_np-cc_aa-seq-counts.tsv.gz',
 '../out/corrected/counts/2024-06-05_r1_np-cc_aa-seq-counts.tsv.gz',
 '../out/corrected/counts/2024-06-05_r0_np-cc_gel-extract_aa-seq-counts.tsv.gz',
 '../out/corrected/counts/2024-06-05_r0_np-cc_repeat_aa-seq-counts.tsv.gz',
 '../out/corrected/counts/2024-06-05_r1_np-cc_gel-extract_aa-seq-counts.tsv.gz']

In [3]:
# read tsv files
df = pd.concat([pd.read_csv(f, sep='\t').assign(round = os.path.basename(f)[11:13]) for f in files], axis=0)

# sum counts within each round
df = df.groupby(['round', 'sequence']).sum().reset_index()

df

,round,sequence,count
0,r0,MAADGYLPDWLEDNLCEGIREWWALKPGAPKPKANQQHQDNARGLV...,1
1,r0,MAADGYLPDWLEDNLCEGIREWWALKPGAPKPKANQQHQDNARGLV...,1
2,r0,MAADGYLPDWLEDNLCEGIREWWALKPGAPKPKANQQHQDNARGLV...,1
3,r0,MAADGYLPDWLEDNLCEGIREWWALKPGAPKPKANQQHQDNARGLV...,1
4,r0,MAADGYLPDWLEDNLCEGIREWWALKPGAPKPKANQQHQDNRRGLV...,1
...,...,...,...
1243361,r1,MAADGYLPDWLEDTLSEGISEWWALKPGVPQPKANQQHQDNRRGLV...,1
1243362,r1,MAADGYLPDWLEDTLSEGISEWWALKPGVPQPKANQQHQDNRRGLV...,1
1243363,r1,MAADGYLPDWLEDTLSEGISEWWKLKPGPPPPKPAERHKDDGRGLV...,1
1243364,r1,MAADGYLPDWLEDTLSEGISEWWKLKPGPPPPKPAERHQDNSRGLV...,1


Count the number of unique sequences and total count in each round.

In [4]:
print("Number of unique sequences in each round:")
df.groupby('round').size()

Number of unique sequences in each round:


round
r0    801140
r1    442226
dtype: int64

In [5]:
print("Total count for each round:")
total = (df
         .drop('sequence', axis=1)
         .groupby('round')
         .sum()
         )
total

Total count for each round:


,count
round,
r0,941365
r1,633141


We have a higher total count for r0 compared to r1.  We will need to take this into account during modelling by weighting the loss function.

Plotting the CDF of counts for the two libraries

## Train/test/eval sets
We will need to remove a dataset for evaluation and testing.  Most sequences have low counts, so we try to get some with high counts in each set. 

For this, we caculate a set of weights according to the model-based enrichment paper, and then include some observations with high and low weights.  The weights are caculated as follows:

$$w_i = \frac{1}{(2\sigma_i^2)}$$

where:

$$\sigma_i^2 = \frac{1}{n_i^{r_1}}(1-\frac{n_i^{r_1}}{\sum_i n^{r_1}_i}) + \frac{1}{n_i^{r_0}}(1-\frac{n_i^{r_0}}{\sum_i n^{r_0}_i})$$

Where $n_i^{r_0}$ is the count for the $i^{th}$ sequence in the round 0 library, and $n_i^{r_1}$ is the count for the $i^{th}$ sequence in the round 1 library.  This value is high if the counts in both libraries are high, and low if the counts in both libraries are low.  First calculating this for each sequence, and plotting the distribution.

We also need to add a pseudo-count of 1 to all observations for this quantitity to be defined.

I also calculate the log enrichment (`le`), which we'll compare against the density ratio for the validation and testing sets.

In [6]:
pseduo = 1
df_unique = (df
             # add pseudocunt
             .assign(count = lambda x: x['count'] + pseduo)
             # r0 and 1 counts in separate columns
             .pivot(index='sequence', columns='round', values='count')
             .fillna(pseduo)
             .reset_index()
             # calculate log enrichment and weights as per MBE paper
             .assign(
                le = lambda x: np.log2((x['r1']/x['r1'].sum())/(x['r0']/x['r0'].sum())),
                sig = lambda x: 1/x['r1']*(1-x['r1']/(x['r1'].sum())) + 1/x['r0']*(1-x['r0']/(x['r0'].sum())),
                weight = lambda x: 1/(2*x['sig']),
                     )
             )
df_unique

round,sequence,r0,r1,le,sig,weight
0,MAADGYLPDWLEDNLCEGIREWWALKPGAPKPKANQQHQDNARGLV...,2.0,1.0,-0.777707,1.499999,0.333334
1,MAADGYLPDWLEDNLCEGIREWWALKPGAPKPKANQQHQDNARGLV...,2.0,1.0,-0.777707,1.499999,0.333334
2,MAADGYLPDWLEDNLCEGIREWWALKPGAPKPKANQQHQDNARGLV...,1.0,2.0,1.222293,1.499999,0.333334
3,MAADGYLPDWLEDNLCEGIREWWALKPGAPKPKANQQHQDNARGLV...,2.0,1.0,-0.777707,1.499999,0.333334
4,MAADGYLPDWLEDNLCEGIREWWALKPGAPKPKANQQHQDNARGLV...,2.0,1.0,-0.777707,1.499999,0.333334
...,...,...,...,...,...,...
1217095,MAADGYLPDWLEDTLSEGISEWWKLKPGPPPPKPAERHKDDGRGLV...,1.0,2.0,1.222293,1.499999,0.333334
1217096,MAADGYLPDWLEDTLSEGISEWWKLKPGPPPPKPAERHKDDSRGLV...,2.0,1.0,-0.777707,1.499999,0.333334
1217097,MAADGYLPDWLEDTLSEGISEWWKLKPGPPPPKPAERHQDNSRGLV...,1.0,2.0,1.222293,1.499999,0.333334
1217098,MAADGYLPDWLEDTLSEGISQWWKLKPGPPPPKPAERHKDDSRGLV...,1.0,2.0,1.222293,1.499999,0.333334


Computing quantiles for the numeric columns will give us an idea of where might be a good threshold for high and low counts.

These weights have a very skewed distribution (as do the counts) - this might be an issue for modelling, since only a few high-count sequences will dominate the results. 

It would be good to try to keep the test and eval sets roughly balanced, with sequences with high and low weights. For this, we will take equal numbers of sequences from the top 10% and the bottom 90%.  We will need to keep these sets small, since otherwise we'll be removing most of the training data. I'll define 'high-confidence' sequences those weights equal to or higher than 1, and 'low-confidence' sequences those with weights lower than 1.  I'll take 50 sequences from each set for each of the eval and test sets.

First, checking how many 'high-confidence' and 'low-confidence' sequences we have with this threshold:

In [7]:
thresh = 1
df_unique = df_unique.assign(set = lambda x: ['high' if w >= thresh else 'low' for w in x['weight']])
df_unique.value_counts('set')

set
low     1212856
high       4244
Name: count, dtype: int64

In [15]:
df_test_eval = df_unique.sample(frac=1).groupby('set').head(100)
df_test = df_test_eval.groupby('set').head(50)

# remove test set from evaluation set
df_eval = df_test_eval[~df_test_eval['sequence'].isin(df_test['sequence'])]
df_train = df_unique[~df_unique['sequence'].isin(df_test_eval['sequence'])]
print("df_train shape: ", df_train.shape)
# check we have 50 sequences in each set, and remainder in training set
df_eval.value_counts('set'), df_test.value_counts('set'), df_train.value_counts('set')

df_train shape:  (1216900, 7)


(set
 high    50
 low     50
 Name: count, dtype: int64,
 set
 high    50
 low     50
 Name: count, dtype: int64,
 set
 low     1212756
 high       4144
 Name: count, dtype: int64)

## Encoding sequences

In this case I one-hot encode the data.  It's more effecient to encode the unique sequences rather than the full training set for model-based enrichment, so I'll do this now and save the results.

Encodign the sequences with labels

In [9]:
max_seq_len = max(len(seq) for seq in df_train['sequence'])
max_seq_len

740

In [10]:
def esm_embedding(seqs, max_len):
    # print(seqs)
    
    # data = [(round_val[0], seqs.loc[round_val[0], 'sequence'][:len(seqs.loc[round_val, 'sequence']) - 1]) for round_val in seqs.index.tolist()]
    data = [(str(round_val), sequence + "<pad>" * (max_len - len(sequence))) for round_val, sequence in seqs]
    # print("DATAA: ", data[0][1])
    model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()

    # print("alphabet: ", alphabet.all_toks)

    batch_converter = alphabet.get_batch_converter(max_len)
    model.eval()

    batch_labels, batch_seq, batch_tokens = batch_converter(data)
    
    # batch_lens is just an array with the length of all of the sequences
    # and alphabet.padding_idx is jus thte index of the token '<pad>' in the alpahabet token list
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
    
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33], return_contacts=True)
    token_representations = results["representations"][33]
    print(token_representations.shape)
    
    # Generate per-sequence representations via averaging
    # # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
    sequence_representations = []
    for i, tokens_len in enumerate(batch_lens):
        sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(1))
    
    print(len(sequence_representations))
    print(len(sequence_representations[0]))
    print(len(sequence_representations[1]))
    
    return batch_labels, sequence_representations


round_sequence_list = [(round_val, df_train.head().loc[round_val, 'sequence'][:len(df_train.head().loc[round_val, 'sequence']) - 1]) for round_val in df_train.head().index.tolist()]
# round_sequence_list[0][1]

embeddings = esm_embedding(round_sequence_list, max_seq_len + 10)
for x in embeddings[1]:
    print(len(x))


torch.Size([5, 752, 1280])
5
737
738
737
738
737
735
737


# Testing a different way to batch the ESM convertions

In [11]:
def esm_embedding(seqs, max_len):
    # print(seqs)
    
    # data is a list of tuples (index, sequence)
    data = [(str(round_val), sequence + "<pad>" * (max_len - len(sequence))) for round_val, sequence in seqs]
    
    model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()

    batch_converter = alphabet.get_batch_converter(max_len)
    model.eval()

    batch_size = 64
    prev = 0
    next = batch_size

    labels = []
    sequence_representations = []
    while next <= len(data) and next != prev:

        batch_labels, batch_seq, batch_tokens = batch_converter(data[prev:next])
        labels = labels + batch_labels
        # batch_lens is just an array with the length of all of the sequences
        # and alphabet.padding_idx is jus thte index of the token '<pad>' in the alpahabet token list
        batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
    
        with torch.no_grad():
            results = model(batch_tokens, repr_layers=[33], return_contacts=True)
        token_representations = results["representations"][33]
        
        # Generate per-sequence representations via averaging
        # # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
        # sequence_representations = []
        for i, tokens_len in enumerate(batch_lens):
            sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(1))

        prev = next
        if next + batch_size > len(data):
            next = len(data)
        else:
            next += batch_size

    
    return labels, sequence_representations


round_sequence_list = [(round_val, df_train.head().loc[round_val, 'sequence'][:len(df_train.head().loc[round_val, 'sequence']) - 1]) for round_val in df_train.head().index.tolist()]
# round_sequence_list[0][1]

embeddings = esm_embedding(round_sequence_list, max_seq_len + 10)
for x in embeddings[1]:
    print(len(x))

embeddings


([], [])

In [13]:
def encode(df, max_len):
    # Extract the index (or this case the round number) and the amino acid sequence of each entry in the data frame
    # index_and_sequence = [(round_val, df.loc[round_val, 'sequence'][:len(df.loc[round_val, 'sequence']) - 1]) for round_val in df.index.tolist()]
    index_and_sequence = [(round_val, df.loc[round_val, 'sequence'].replace('*', '<eos>')) for round_val in df.index.tolist()]
    indexes, encoded_tensor = esm_embedding(index_and_sequence, max_len)

    numpy_arrays = [tensor.numpy() for tensor in encoded_tensor]
    
    df_encoded = pd.DataFrame({
        'round': indexes,
        'encoded': numpy_arrays
    })

    df_encoded.set_index('round', inplace=True)
    df_encoded.index = df_encoded.index.astype('int64')

    out = df
    # out['encoded'] = df_encoded
    out.loc[:,'encoded'] = df_encoded
    
    return out

df_train_test = encode(df_train.head(), 750)

# print(df_train_test['encoded'][0])
df_train_test['encoded'][0].shape
for x in df_train_test['encoded']:
    print(len(x))

/tmp/ipykernel_914831/309154787.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out.loc[:,'encoded'] = df_encoded


TypeError: object of type 'float' has no len()

In [14]:
print(df_train.shape)

(1216900, 7)


In [60]:
max_seq_length = df_unique['sequence'].str.len().max() + 10
# rerun_encoding = True

if not os.path.exists(os.path.join(procdir, 'train_label_encoding.pkl')) or rerun_encoding:
    print("recoding")
    
    df_train = encode(df_train, max_seq_length)
    print('after df_train')
    df_eval = encode(df_eval, max_seq_length)
    print("after df_eval")
    df_test = encode(df_test, max_seq_length)
    
    # save data
    df_train.to_pickle(os.path.join(procdir, 'train_ESM_embedding.pkl'))
    df_eval.to_pickle(os.path.join(procdir, 'eval_ESM_embedding.pkl'))
    df_test.to_pickle(os.path.join(procdir, 'test_ESM_embedding.pkl'))

else:
    
    #df_train = pd.read_pickle(os.path.join(procdir, 'train_ESM_embedding.pkl')) # too large
    df_eval = pd.read_pickle(os.path.join(procdir, 'eval_ESM_embedding.pkl'))
    df_test = pd.read_pickle(os.path.join(procdir, 'test_ESM_embedding.pkl'))
    # max_seq_length would not be defined if the dfs were already loaded in an external file


recoding


## Model-based enrichment dataset

For model based enrichment, we create a dataset where we sample sequences the number of times they were seen in each selection round.  For each sample, the sequence is labelled according to which round it appeared in: -1 for r0 and +1 for r1.  For example, if we have a sequence that was seen once in r0 and twice in r1, it would appear in the final dataset once with a label of -1 and twice with a label of +1.

Note that the paper describes using class labels of -1 and +1, and they say they use 'standard logistic loss'.  However, the loss function they describe looks slightly different to the [`BCELoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html) in pytorch, probably mostly because the pytorch version expects class labels of 0 and 1 (which are more standard).  Since it's easier (and probably much faster) to the use BCELoss function from pytorch, I'll also use class labels of 0 and 1.

The evaluation set would look like this:

In [ ]:
# add labels to data based on if the sequence base observed in the r0 and r1 libraries
df_example = (df_eval
                .loc[:, ['encoded', 'r0', 'r1']]
                .melt(id_vars = 'encoded', value_vars = ['r0', 'r1'], var_name = 'round', value_name = 'count')
                .assign(label = lambda x: (x['round'] == 'r1').astype(int))
                )

# sample sequences according to their count
df_example = df_example.reindex(df_example.index.repeat(df_example['count']))

df_example

,encoded,round,count,label
0,"[10.0, 0.0, 0.0, 2.0, 5.0, 19.0, 9.0, 12.0, 2....",r0,2.0,0
0,"[10.0, 0.0, 0.0, 2.0, 5.0, 19.0, 9.0, 12.0, 2....",r0,2.0,0
1,"[10.0, 0.0, 0.0, 2.0, 5.0, 19.0, 9.0, 12.0, 2....",r0,1.0,0
2,"[10.0, 0.0, 0.0, 2.0, 5.0, 19.0, 9.0, 12.0, 2....",r0,1.0,0
3,"[10.0, 0.0, 0.0, 2.0, 5.0, 19.0, 9.0, 12.0, 2....",r0,1.0,0
...,...,...,...,...
199,"[10.0, 0.0, 0.0, 2.0, 5.0, 19.0, 9.0, 12.0, 2....",r1,7.0,1
199,"[10.0, 0.0, 0.0, 2.0, 5.0, 19.0, 9.0, 12.0, 2....",r1,7.0,1
199,"[10.0, 0.0, 0.0, 2.0, 5.0, 19.0, 9.0, 12.0, 2....",r1,7.0,1
199,"[10.0, 0.0, 0.0, 2.0, 5.0, 19.0, 9.0, 12.0, 2....",r1,7.0,1


## Dataloader

Creating a dataset and dataloader class for these datasets.  I wrap some of the code from earlier in a lightning data module class.

In [20]:
class MBEDataset(Dataset):

    def __init__(self, df, file_name):
        self.df_raw = df
        self.df = self.create_mbe_dataset(df)
        self.ddf = dd.read_parquet(os.path.join(procdir, file_name))
    
    def create_mbe_dataset(self, df):

        df = (df
              .loc[:, ['encoded', 'r0', 'r1']]
              .melt(id_vars=['encoded'], value_vars=['r0', 'r1'], var_name='round', value_name='value')
              .assign(label = lambda x: (x['round'] == 'r1').astype(int))
        )
        df = df.reindex(df.index.repeat(df['value'])).reset_index()

        return df

    def get_weights(self):
        """
        Number of examples in each class
        """
        counts = (self.df
                .groupby('label')
                .size()
               )
        return torch.tensor(counts.loc[1] / counts.loc[0], dtype=torch.float64)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # TODO: instead of getting it from the data set, get it from the file...
        # row = self.df.iloc[idx]
        row = self.ddf.loc[idx].compute()
        return row['encoded'], float(row['label'])

class LEDataset(Dataset):
    """
    Log enrichment dataset
    """

    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # TODO: instead of getting it from the data set, get it from the file...
        row = self.df.iloc[idx]
        return row['encoded'], row['le']

class MBEDataModule(L.LightningDataModule):

    def __init__(self, data_dir, batch_size = 32):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

    def setup(self, enc="ESM_embedding", stage=None):
        self.train = pd.read_pickle(os.path.join(self.data_dir, f'train_{enc}.pkl'))
        self.eval = pd.read_pickle(os.path.join(self.data_dir, f'eval_{enc}.pkl'))
        self.test = pd.read_pickle(os.path.join(self.data_dir, f'test_{enc}.pkl'))

    def train_dataloader(self):
        return DataLoader(MBEDataset(self.train, 'train_ESM_embedding.pkl'), batch_size=self.batch_size, shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(LEDataset(self.eval), batch_size=self.batch_size)
    
    def test_dataloader(self):
        return DataLoader(LEDataset(self.test), batch_size=self.batch_size)
    
# test out dataset class
ds = MBEDataset(df_eval, 'eval_label_encoding.pkl')
ds[0]



(array([10.,  0.,  0.,  2.,  5., 19.,  9., 12.,  2., 18.,  9.,  3.,  2.,
        16.,  9., 15.,  3.,  5.,  7., 14., 13., 18., 18.,  8.,  9.,  8.,
        12.,  5., 12., 12., 12., 12.,  8., 12.,  0.,  3., 14.,  6.,  8.,
         2.,  2., 15., 14.,  5.,  9., 17.,  9., 12.,  5., 19.,  8., 19.,
         9.,  5., 12.,  4., 11.,  5.,  9.,  2.,  8.,  5.,  3., 12., 17.,
        11.,  3.,  0.,  2.,  0.,  0.,  0.,  9.,  3.,  6.,  2.,  8.,  0.,
        19.,  2., 14., 13.,  9.,  2., 15.,  5.,  2., 11., 12., 19.,  9.,
         8., 19., 11.,  6.,  0.,  2.,  0.,  3.,  4., 13.,  3., 14.,  9.,
         8.,  3.,  2., 16., 15.,  4.,  5.,  5., 11.,  9.,  5., 14.,  0.,
        17.,  4., 13.,  0.,  8.,  8., 14., 17.,  9.,  3., 12.,  9.,  5.,
         9., 17.,  3.,  3.,  0.,  0.,  8., 16.,  0., 12.,  5.,  8.,  8.,
        14., 12., 17.,  2., 13., 15., 12., 13.,  3., 12.,  2., 15., 15.,
        15.,  5., 17.,  5.,  8., 15.,  5.,  8., 13., 12.,  0., 14.,  8.,
        14.,  9., 11.,  4.,  5., 13., 16.,  5.,  2.

In [21]:
ds.get_weights()

tensor(1.0544, dtype=torch.float64)

In [ ]:
le = LEDataset(df_eval)
le[0]

(array([10.,  0.,  0.,  2.,  5., 19.,  9., 12.,  2., 18.,  9.,  3.,  2.,
        16.,  9., 15.,  3.,  5.,  7., 14., 13., 18., 18.,  8.,  9.,  8.,
        12.,  5., 12., 12., 12., 12.,  8., 12.,  0.,  3., 14.,  6.,  8.,
         2.,  2., 15., 14.,  5.,  9., 17.,  9., 12.,  5., 19.,  8., 19.,
         9.,  5., 12.,  4., 11.,  5.,  9.,  2.,  8.,  5.,  3., 12., 17.,
        11.,  3.,  0.,  2.,  0.,  0.,  0.,  9.,  3.,  6.,  2.,  8.,  0.,
        19.,  2., 14., 13.,  9.,  2., 15.,  5.,  2., 11., 12., 19.,  9.,
         8., 19., 11.,  6.,  0.,  2.,  0.,  3.,  4., 13.,  3., 14.,  9.,
         8.,  3.,  2., 16., 15.,  4.,  5.,  5., 11.,  9.,  5., 14.,  0.,
        17.,  4., 13.,  0.,  8.,  8., 14., 17.,  9.,  3., 12.,  9.,  5.,
         9., 17.,  3.,  3.,  0.,  0.,  8., 16.,  0., 12.,  5.,  8.,  8.,
        14., 12., 17.,  2., 13., 15., 12., 13.,  3., 12.,  2., 15., 15.,
        15.,  5., 17.,  5.,  8., 15.,  5.,  8., 13., 12.,  0., 14.,  8.,
        14.,  9., 11.,  4.,  5., 13., 16.,  5.,  2.

In [ ]:
# test out datamodule class - takes a while to load pickled data
dm = MBEDataModule(procdir, batch_size=BATCH_SIZE)
dm.setup()
test = next(iter(dm.train_dataloader()))
test

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64),
 tensor([1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1.,
         1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
         1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0.,
         0., 1., 1., 1., 0., 1., 1., 1., 0., 1.], dtype=torch.float64)]

## Model

In this case, we'll use a simple feedforward network with two layers.

In [21]:
class MBELogisticRegression(nn.Module):
    """
    Class for logistic regression. 
    Returns logits by default, or probabilities if probs=True.
    """

    def __init__(self, input_size, pos_weight=None):
        super().__init__()
        self.linear = nn.Linear(input_size, 1, dtype = torch.float64)
        self.pos_weight = 1 if pos_weight is None else pos_weight

    def forward(self, x, probs=False):
        """
        Return logits or probabilities
        
        """

        logits = self.linear(x)
        if probs:
            return torch.sigmoid(logits)
        else:
            return logits

    def predict(self, x):
        """
        Return density ratio, an estimate of log enrichment
        """
        
        # get probability for positive class
        p = self.forward(x, probs=True)

        # density ratio is p/(1-p), adjusted for class imbalance
        return p/(1-p) / self.pos_weight

class MBEFeedForward(MBELogisticRegression):

    def __init__(self, input_size, pos_weight=None, n_units=128):
        super().__init__(input_size, pos_weight)
        self.n_units = n_units
        self.pos_weight = pos_weight
        self.linear = nn.Sequential(
            nn.Linear(input_size, n_units, dtype = torch.float64),
            nn.ReLU(),
            nn.Linear(n_units, 1, dtype = torch.float64),
        )


model = MBEFeedForward(len(ds[0][0]), pos_weight=ds.get_weights())
model(test[0])[:5]

tensor([[ 0.0265],
        [ 0.0588],
        [ 0.0310],
        [ 0.0127],
        [-0.0194]], dtype=torch.float64, grad_fn=<SliceBackward0>)

In [22]:
model.predict(test[0])[:5]

tensor([[0.9739],
        [1.0059],
        [0.9783],
        [0.9605],
        [0.9302]], dtype=torch.float64, grad_fn=<SliceBackward0>)

Use a weighted BCE loss function with the Adam optimizer.  The classes are weighted using the ratio of total reads in r1 vs r0.

In [23]:
class LitMBE(L.LightningModule):

    def __init__(self, model, lr=0.001, pos_weight = 1):
        super().__init__()
        self.model = model
        self.loss = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
        self.spearman = stats.spearmanr 
        self.lr = lr
        self.save_hyperparameters()

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.model(x).squeeze()
        loss = self.loss(logits, y)
        self.log('train_loss', loss, on_step = True, on_epoch = True, prog_bar = True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        with torch.no_grad():
            le = self.model.predict(x).squeeze().cpu().numpy()
            spearman = self.spearman(le, y.cpu().numpy()).statistic
            self.log('val_spearman', spearman, on_step = False, on_epoch = True, prog_bar = True)
        

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=0.001)

In [24]:
# instantiate lightning model
lit_model = LitMBE(model, pos_weight=ds.get_weights())

# use weights and biases logger
wandb_logger = WandbLogger(project='mbe', name = "feedforward")
wandb_logger.experiment.config.update({
    "lr": 0.001,
    "pos_weight": ds.get_weights(),
    "n_units": 128,
    "batch_size": BATCH_SIZE,
    "max_seq_length": max_seq_length,
    "arch": "feedforward",
    "enc": "onehot",
    "loss": "BCEWithLogitsLoss",
    "opt": "Adam"
})

# train model
trainer = L.Trainer(max_epochs=10, logger = wandb_logger)
trainer.validate(model = lit_model, datamodule = dm)
trainer.fit(model = lit_model, datamodule = dm)
wandb.finish()

/data/home/tlorant@cmri.com.au/aavolve_data/.micromamba/envs/default/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A10') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/data/home/tlorant@cmri.com.au/aavolve_data/.micromamba/envs/default/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=95` in the `DataLoader` to improve performance.


Validation: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_spearman         -0.008669466711580753
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type              | Params
--------------------------------------------
0 | model | MBEFeedForward    | 2.0 M 
1 | loss  | BCEWithLogitsLoss | 0     
--------------------------------------------
2.0 M     Trainable params
0         Non-trainable params
2.0 M     Total params
8.065     Total estimated model params size (MB)


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
train_loss_epoch,█▁▃▃▃▂▁▂▃▂
train_loss_step,▇▅▃▃▃▄▄█▄▅▄▅▃▄▅▃▅▅▃▄▆▄▄▆▃▄▄▆▇▅▃▄▄▄▄▂▅▄▂▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_spearman,▁█▇█▆▇▇▇▇▇▇
epoch,9
train_loss_epoch,0.70579
train_loss_step,0.72748
trainer/global_step,626039
val_spearman,0.36864


Future improvements / things to try:

 - Weight decay / regularization
 - Different architechtures
 - Different encoding (label encoding / embeddings from ESM-2)
 - Weight decay / regularization